# Vector spaces and linear transforms

In [ ]:
# Standard imports
# Make sure you run this cell!
# NumPy
import numpy as np  

# Set up Matplotlib
import matplotlib as mpl   
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('figure', figsize=(8.0, 4.0), dpi=140)

# custom utils
import utils.image_audio as ia

print("Everything imported OK")

In [ ]:
import matplotlib as mpl   
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('figure', figsize=(8.0, 4.0), dpi=140)


## Guidance
You will need to understand the following functions well to complete this exercises:
* [`np.argmin()`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.argmin.html)
* [`np.argsort()`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.argsort.html)
* [`np.linalg.norm()`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.norm.html)
* [`np.cov()`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.cov.html)

## Whisky: Representing and comparing vectors

Whisky distillation is a major industry in Scotland. 

The dataset in `data/whisky.csv` is data from a number of whisky distilleries. For each distillery, there are a set of subjective judgements about the flavour characteristics of their production. The data comes from [this Strathclyde University research project](https://www.mathstat.strath.ac.uk/outreach/nessie/nessie_whisky.html).

Each distillery has been judged on twelve flavour indicators (like "smokiness" or "sweetness"), and they have been assigned values from 0-3 indicating the strength of that category, judged by an expert whisky drinker. These can be seen as 12D vectors, one vector per distillery. **Every distillery is represented as a point in twelve dimensional vector space.**

We also have a 2D array of the geographical locations of each distillery. The code below loads the data.

## Loading the data

In [ ]:
## Thic code contains pandas dataframe code but you don't
# need to understand it to complete this exercise. 
# If you wish to learn about dataframes, ask a tutor or lookup the official documentation.

import pandas as pd
whisky_df = pd.read_csv("data/whiskies.txt")
whisky_df = whisky_df.sort_values(by='Distillery')
# extract the column and row names
distilleries= np.array(whisky_df["Distillery"])

columns = {name.lower():index for index,name in enumerate(whisky_df.columns[2:-3])}

# split apart the data frame and form numpy arrays
locations = np.array(whisky_df.iloc[:,-2:])
whisky = np.array(whisky_df.iloc[:,2:-3])

# fix wine column
columns['wine'] = columns['winey']
del columns['winey']
# force tie breaks 
np.random.seed(2018)
whisky = whisky + np.random.normal(0,0.1,whisky.shape)

## Viewing the data

We can see this whole dataset as a heatmap:

In [ ]:
# show a plot of the whisky data
fig = plt.figure(figsize=(6,15))
ax = fig.add_subplot(1,1,1)

# image plot
img = ax.imshow(whisky)
ax.set_yticks(np.arange(len(distilleries)))
ax.set_yticklabels(distilleries, rotation="horizontal", fontsize=6)

# put the x axis at the top
ax.xaxis.tick_top()
ax.set_xticks(np.arange(len(columns)))
ax.set_xticklabels(columns, rotation="vertical", fontsize=6)


### Available data
You now have these variables:

* `whisky` an  86x12 array of taste judgements, one row for each of the 86 distilleries. Each whisky has a rating 0-3 for each of the 12 flavour categories.
* `distilleries` is a list of 86 distillery names
* `columns` is a mapping of feature names to column indices.
* `locations` is an 86x2 matrix of positions of each distillery in [OS grid reference format](https://www.gridreferencefinder.com/) in the same order as `whisky`

For example:

In [ ]:
print(whisky[distilleries.searchsorted('Glenfiddich'), columns['smoky']])

will tell you how "smoky" Glenfiddich was rated.

In [ ]:
print(distilleries[8])

will tell you the 9th distillery in the dataset is `Aultmore`.

In [ ]:
print(locations[distilleries.searchsorted('Glengoyne')])

## A: Norms, interpolation and statistics
We will see some simple things we can do with this dataset. 


We can compute distances in **flavour space** between distilleries.

For example, we can compute the distance between the `Glenlivet` distillery and every other distillery *in terms of flavour, not physical distance*. The result will be a 1D vector of 86 distances. 

Remember: distance of $\vec{x}$ and $\vec{y}$ is the norm of their difference: $\|\vec{x}-\vec{y}\|$

We can compute this for several different norms (e.g. $L_1, L_2$, and $L_\infty$)

We need to compute the norm by telling `np.linalg.norm` to compute the lengths of vectors over columns (`axis=1`), as we have an $N\times D$ matrix: $N$ rows of observations (distilleries), each with $D$ measurements (flavour ratings).

In [ ]:
# Get the flavout vector for Glenlivet
glenlivet = whisky[distilleries.searchsorted('Glenlivet')]
print(glenlivet)

In [ ]:
## Compute distances
# must use axis=1 to get the right result
# otherwise a matrix norm will be used!
glenlivet_2 = np.linalg.norm(whisky - glenlivet, 2, axis=1)
glenlivet_1 = np.linalg.norm(whisky - glenlivet, 1, axis=1)
glenlivet_inf = np.linalg.norm(whisky - glenlivet, np.inf, axis=1)

### Visualising these distances
We can visualise these distances. This lets us see which distilleries produce whisky most similar to Glenlivet and which of them are most dissimilar. Note that we can use `argsort` to order a list of distances. Below, we plot a faceted graph, one facet for each norm, showing each distillery's flavour distance to `Glenlivet` as a rank bar plot. The labels are a bit small, but the graphs are a useful summary of the distances in this abstract 12 dimensional space.

In [ ]:
fig = plt.figure(figsize=(8,12))

# you can use this utility function to
# help you get the xticklabels in order
def list_in_order(alist, order):
    """Given a list 'alist' and a list of indices 'order'
    returns the list in the order given by the indices"""
    return [alist[i] for i in order]
    

ax = fig.add_subplot(3,1,1)

def rank_plot(distances):
    # find the ordering of the distances
    order = np.argsort(distances)
    # bar plot them
    ax.bar(np.arange(len(distances)), distances[order])           
    ax.set_xlabel("Distillery")
    ax.set_ylabel("Distance to Glenlivet (in flavour space)")
    ax.set_xticks(np.arange(86))    
    ax.set_frame_on(False)
    # make sure the same order is used for the labels!
    ax.set_xticklabels(list_in_order(distilleries, order),
                       rotation="vertical", fontsize=4)

# make the plots
ax = fig.add_subplot(3,1,1)
ax.set_title("$L_2$ norm")
rank_plot(glenlivet_2)
ax = fig.add_subplot(3,1,2)
ax.set_title("$L_1$ norm")
rank_plot(glenlivet_1)
ax = fig.add_subplot(3,1,3)
ax.set_title("$L_\infty$ norm")
rank_plot(glenlivet_inf)

# removes ugly overlapping
plt.tight_layout()


## Tasks

**Task A.1** 

Compute the $L_2$ norm of each distillery's flavour vector from the flavour vector corresponding to the distillery `Auchentoshan`, and store it in `auchentoshan_2`

In [ ]:
#code goes here

**Task A.2** 
Which distillery is furthest from Auchentoshan's flavour profile in the $L_2$ norm? Store the distillery name in the variable in `unlike_auchentoshan`. Note: compute this! No hardcoding!


In [ ]:
#code goes here

**Note** from now on, use the L2 norm if you need to compute any norms.


### Vector arithmetic
A client says to you:
    
>    I'd like something a bit like Balvenie. But a bit more "lively", the way Cragganmore is more "lively" than Springbank.

Which whisky should you recommend? 

We can work this out:
* What does more "lively" mean? We don't have a "lively" column.
    * But we do have a reference point: Springbank->Cragganmore is somehow "lively"
    * This "direction" between these flavour vectors is *also* a vector
* How do we combine Balvenie's flavour profile with "lively"? We can compose vectors by addition.
* How do we find a distillery that represents this profile? We can compute lengths of vectors using a norm.

**Note: this question is not subjective, nor does it require any trial-and-error or knowledge about whisky. Answer it directly using vector arithmetic.**



In [ ]:
#code goes here

**Task A.3** Find the distillery that best represents (L2 norm) a flavour profile halfway between `Lagavulin` and `Bowmore`. Store the name in a variable `halfway_whisky`.

In [ ]:
#code goes here

## Region flavours

We could say that the "representative" element of a collection of vectors was the one closest to the geometric centroid. This is given by the **mean vector** of a data set.

**Task A.4** Compute the mean vector of all of the flavour vectors. Use it to find the names of two distilleries:
* `most_representative` The distillery with the **most** representative flavour profile
* `least_representative` The distillery with the **least** representative flavour profile

In [ ]:
#code goes here


### A map
The code below will show a map of Scotland, with the distilleries in their correct positions. Different geographic regions have different characteristic flavour profiles.

One very distinctive region is **Speyside**. This is a region bounded roughly by the box

    263000,860000 -> 343000,785000
    
in the same OS grid units used in the `locations` array. This is highlighted on the map below.


In [ ]:
from whisky_map import draw_map, map_box

# draw each distillery label at the locations given.
ax = draw_map(locations, distilleries)

# show the Speyside region
# draw a box in OS grid units
map_box(ax, 263000, 785000, 343000,860000 )

**Task A.5**
* Find all distilleries in Speyside and use this to answer these questions:
* `speyside_flavour`: Compute the most typical whisky flavour profile for Speyside. (i.e. a 12 element vector).
* `most_typical_speyside`: The specific name of the distillery from Speyside that is most typical of that region;
* `least_typical_speyside`: The specific name of the distillery from Speyside that is least typical of that region;
* `most_like_speyside`: The specific name of the distillery from **outside** Speyside that is most similar to a typical Speyside distillery.


**Note:** do not do any of this by hand. Write code.
Hint: Boolean arrays.

In [ ]:
#your code goes here

In [ ]:
print("The most typical whisky of Speyside is {most}.".format(most=most_typical_speyside))
print("The least typical Speyside whisky of the style is {least}.".format(least=least_typical_speyside))
print("The non-Speyside whisky most like a Speyside whisky is {nonspeyside}.".
      format(nonspeyside=most_like_speyside))        

## Demeaning
The dataset that we have is unnormalised. It is a set of ratings, 0-4, and many of the attributes rated are very correlated (e.g. `smoky` and `medicinal`). Many of the ratings are also on quite different scales, with `tobacco` being much less likely to be rated 4 that `floral`.

It is easier to work with normalised data. 

**Task B.1**
Compute `demeaned_whisky` that has the mean flavour vector removed. 



In [ ]:
#code goes here

* **Task B.2**
Use `np.cov()` to compute the **covariance matrix** of the demeaned data. Call this **whisky_cov**. The code below will show you this as an image. **Note**: this should be a 12x12 matrix!

This tells us how different columns of the dataset are correlated (co-vary) with each other.

We can see several interesting things:
* having `body` is negatively correlated with the `wine` flavour
* being `smoky` is negatively correlated with having `sweetness`
* `nutty` is positively correlated with `body`

Although the raters have provided 12 different categories, it seems these are not fully independent of each other.


In [ ]:
#code goes here

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
img = ax.imshow(whisky_cov, cmap='seismic', vmin=-1.5, vmax=1.5)
ax.set_xticks(np.arange(12))
ax.set_yticks(np.arange(12))
ax.set_yticklabels(columns)
ax.set_xticklabels(columns, rotation='vertical')
ax.set_title("Covariance matrix")
fig.colorbar(img)
